#### Import Libraries

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


#### Data Reading

In [0]:
df_customers = spark.readStream.format('delta') \
                    .load('abfss://bronze@dldatabricks.dfs.core.windows.net/customers')


In [0]:
# df_customers.display()

In [0]:
df_customers = df_customers.withColumnRenamed('_rescued_data','rescued_data') \
                            .withColumn('domain', split(col('email'),'@')[1]) \
                            .withColumn('full_name', concat_ws(' ', col('first_name'), col('last_name'))) \
                            .select('customer_id','full_name','email','domain','city','state')
# df_customers.display()


#### Data Writing

In [0]:
df_customers.writeStream.format('delta') \
            .outputMode('append') \
            .option('checkpointLocation', 'abfss://silver@dldatabricks.dfs.core.windows.net/checkpoint_customers') \
            .option('path','abfss://silver@dldatabricks.dfs.core.windows.net/customers') \
            .trigger(once=True) \
            .start()

#### Data Aggregations

In [0]:
# df = spark.read.format('delta').load('abfss://silver@dldatabricks.dfs.core.windows.net/customers')
# df.display()

In [0]:
# df_domain_grp = df.groupBy('domain').agg(count('customer_id').alias('total_customers')) \
#             .sort('total_customers',ascending = False)
# display(df_domain_grp)

In [0]:
# df_state_grp = df.groupBy('state').agg(count('customer_id').alias('total_customers')) \
#                     .sort('total_customers', ascending = False)
# display(df_state_grp)